In [1]:
import pandas as pd
import numpy as np

#plotting
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#model building
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
matches_data = pd.read_csv('/Users/prathyushachelemella/Downloads/games.csv')

columns_to_drop = ['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'TEAM_ID_home', 'FG_PCT_home', 'FT_PCT_home', 
                   'FG3_PCT_home', 'TEAM_ID_away','FG_PCT_away', 'FT_PCT_away','FG3_PCT_away']

matches_data = matches_data.drop(columns_to_drop, axis=1)
matches_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,AST_home,REB_home,PTS_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,1610612740,1610612759,2022,126.0,25.0,46.0,117.0,23.0,44.0,1
1,22200478,1610612762,1610612764,2022,120.0,16.0,40.0,112.0,20.0,37.0,1
2,22200466,1610612739,1610612749,2022,114.0,22.0,37.0,106.0,20.0,46.0,1
3,22200467,1610612755,1610612765,2022,113.0,27.0,49.0,93.0,15.0,46.0,1
4,22200468,1610612737,1610612741,2022,108.0,22.0,47.0,110.0,20.0,47.0,0


In [3]:
matches_data.shape

(26651, 11)

In [4]:
matches_data.columns

Index(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'PTS_home',
       'AST_home', 'REB_home', 'PTS_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS'],
      dtype='object')

In [5]:
len(matches_data)

26651

In [6]:
list1 = matches_data.columns.to_list()
remove_from_list = ['REB_home', 'REB_away']

for i in range(len(remove_from_list)):
    list1.remove(remove_from_list[i])
    
for i in list1:
    print('The unique values', i, 'are: ', matches_data[i].unique())

The unique values GAME_ID are:  [22200477 22200478 22200466 ... 11400005 11400002 11400001]
The unique values HOME_TEAM_ID are:  [1610612740 1610612762 1610612739 1610612755 1610612737 1610612738
 1610612751 1610612752 1610612745 1610612750 1610612760 1610612758
 1610612746 1610612765 1610612748 1610612756 1610612743 1610612754
 1610612761 1610612747 1610612759 1610612749 1610612766 1610612741
 1610612742 1610612763 1610612753 1610612764 1610612757 1610612744]
The unique values VISITOR_TEAM_ID are:  [1610612759 1610612764 1610612749 1610612765 1610612741 1610612754
 1610612744 1610612761 1610612753 1610612742 1610612757 1610612747
 1610612766 1610612762 1610612763 1610612752 1610612751 1610612748
 1610612740 1610612737 1610612758 1610612750 1610612743 1610612756
 1610612739 1610612738 1610612760 1610612746 1610612745 1610612755]
The unique values SEASON are:  [2022 2021 2020 2019 2013 2012 2011 2010 2009 2008 2007 2006 2005 2004
 2003 2018 2017 2016 2015 2014]
The unique values PTS_hom

In [7]:
win_df=matches_data.groupby(['GAME_ID','SEASON']).sum()['HOME_TEAM_WINS'].reset_index()

win_df

,GAME_ID,SEASON,HOME_TEAM_WINS
0,10300001,2003,1
1,10300002,2003,1
2,10300003,2003,0
3,10300004,2003,1
4,10300005,2003,1
...,...,...,...
26617,52100111,2021,1
26618,52100121,2021,1
26619,52100131,2021,1
26620,52100201,2021,0


In [8]:
win_df = win_df[win_df['HOME_TEAM_WINS']==1]
win_df['SEASON'] = win_df['SEASON'].apply(lambda x:x+1)#to get target
win_df

<ipython-input-8-2ee73ae2e93d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_df['SEASON'] = win_df['SEASON'].apply(lambda x:x+1)#to get target


,GAME_ID,SEASON,HOME_TEAM_WINS
0,10300001,2004,1
1,10300002,2004,1
3,10300004,2004,1
4,10300005,2004,1
6,10300007,2004,1
...,...,...,...
26614,52000201,2021,1
26616,52100101,2022,1
26617,52100111,2022,1
26618,52100121,2022,1


In [9]:
match_df = matches_data.merge(win_df[['GAME_ID','HOME_TEAM_WINS']],
                       left_on='HOME_TEAM_ID',right_on='GAME_ID')

match_df

,GAME_ID_x,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,AST_home,REB_home,PTS_away,AST_away,REB_away,HOME_TEAM_WINS_x,GAME_ID_y,HOME_TEAM_WINS_y


In [10]:
match_df['HOME_TEAM_ID'].unique()

array([], dtype=int64)

In [11]:
match_df[match_df['AST_home']==1].style.background_gradient(cmap = 'plasma')
match_df

,GAME_ID_x,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,AST_home,REB_home,PTS_away,AST_away,REB_away,HOME_TEAM_WINS_x,GAME_ID_y,HOME_TEAM_WINS_y


In [12]:
match_df = match_df.dropna()

match_df.isnull().sum()

GAME_ID_x           0.0
HOME_TEAM_ID        0.0
VISITOR_TEAM_ID     0.0
SEASON              0.0
PTS_home            0.0
AST_home            0.0
REB_home            0.0
PTS_away            0.0
AST_away            0.0
REB_away            0.0
HOME_TEAM_WINS_x    0.0
GAME_ID_y           0.0
HOME_TEAM_WINS_y    0.0
dtype: float64

In [13]:
data = match_df.copy()

from sklearn.model_selection import train_test_split
X = matches_data.drop(['HOME_TEAM_WINS'], axis=1)
y = matches_data['HOME_TEAM_WINS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train.shape,X_test.shape
X_train.columns

Index(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'PTS_home',
       'AST_home', 'REB_home', 'PTS_away', 'AST_away', 'REB_away'],
      dtype='object')

In [14]:
cf = ColumnTransformer(transformers = [
    ('tnf1',OneHotEncoder(sparse=False,drop='first'),['HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON'])
],remainder='passthrough')

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Create a pipeline with a SimpleImputer and a LogisticRegression classifier
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', LogisticRegression())
])

# Fit the pipeline to your data
pipeline.fit(X_train, y_train)

# Make predictions using the trained pipeline
y_pred = pipeline.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))

pipeline.predict_proba(X_test)[10]

0.5871318701932096


array([0.41364844, 0.58635156])

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


pipeline2 = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', RandomForestClassifier())
])

pipeline2.fit(X_train,y_train)
y_pred = pipeline2.predict(X_test)
print(metrics.accuracy_score(y_test,pipeline2.predict(X_test)))

pipeline2.predict_proba(X_test)[10]



0.9968111048583755


array([1., 0.])

In [17]:
import pickle
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))

In [20]:
def predict_game(team_home, team_away, model):
    # Load team and opponent statistics
    home_stats = load_team_stats(team_home)
    away_stats = load_team_stats(team_away)

    # Combine team and opponent statistics
    X = np.hstack((home_stats, away_stats))

    # Make a prediction using the loaded model
    y_pred = model.predict(X)

    # Calculate the predicted score for the home team
    PTS_home = round(y_pred[0])

    # Return the predicted score for the home team
    return PTS_home

In [23]:
import streamlit as st
import pandas as pd
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Declaring the teams

teams = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets',
         'Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets',
         'Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers',
         'Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat',
         'Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans',
         'New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers',
         'Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs',
         'Toronto Raptors','Utah Jazz','Washington Wizards']

# declaring the venues where the matches are going to take place

season = ['2003','2004','2005','2006','2007','2008','2009','2010','2011','2012',
'2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']


# Loading our machine learning model from a saved pickle file

pipeline = pickle.load(open('pipeline.pkl', 'rb')) #remember all folders including pipe.pkl,
# notebook, datasets exist in the same directory

# Setting up the app's title

st.title('NBA Match Predictor')


# Setting up the layout with two columns
col1, col2 = st.columns(2)

# Creating a dropdown selector for the batting team
with col1:
    hometeam = st.selectbox('Select the home team', sorted(teams))

# Creating a dropdown selector for the bowling team
with col2:

    visitorteam = st.selectbox('Select the visitor team', sorted(teams))


# Creating a dropdown selector for the city where the match is being played
season = st.selectbox(
    'Select the season', sorted(season))

# Creating a numeric input for the target score using number_input method in streamlit
target = int(st.number_input('Target', step=1))

# Setting up the layout with three columns
col3, col4, col5 = st.columns(3)

# Creating a numeric input for the current score
with col3:
    win = int(st.number_input('win', step=1))
PTS_home = round(y_pred[0])
# Checking for different match results based on the input provided
if PTS_home > target:
    st.write(hometeam,"won the match")
    
elif PTS_home == target-1 and overs==20:
    st.write("Match Drawn")
    
    
st.write('To proceed, please select different teams because no match can be played \
without two different teams.')

try:

    if st.button('Predict Probability'):
                
                               
                # Creating a pandas DataFrame containing the user inputs
                input_df = pd.DataFrame(
                               {'home_team': [hometeam], 
                                'visitor_team': [visitorteam], 
                                'season': [season], 
                                })
                # Loading the trained machine learning pipeline to make the prediction
                result = pipe.predict_proba(input_df)
                
                # Extracting the likelihood of loss and win
                lossprob = result[0][0]
                winprob = result[0][1]
                
                # Displaying the predicted likelihood of winning and losing in percentage

                st.header(hometeam+"- "+str(round(winprob*100))+"%")

                st.header(visitorteam+"- "+str(round(lossprob*100))+"%")
                
                
        #Catching ZeroDivisionError         
except ZeroDivisionError:
    st.error("Please fill all the details")
            
    #Displaying an error message if the input is incorrect        
else:
    st.error('There is something wrong with the input, please fill the correct details')

In [ ]:
pip freeze > requirements.txt